In [1]:
# -*- coding: utf-8 -*-

import os
import re
import sys
import json
import redfish # Module sources at: https://github.com/DMTF/python-redfish-library/blob/master/src/redfish/rest/v1.py

###########################################################################################################
#
# Variables
#
###########################################################################################################

# Set Student ID number
Stud="00"
print ("You are Student" + Stud + "\n\n")

# iLO common credentials
user="student"
Password = "P@ssw0rd!"

# iLO IP addresses with port

iLO5IP = "ilo5"
base_url = "https://" + iLO5IP

You are Student00




In [14]:
###########################################################################################################
#
# iLO 5 Devices
#
###########################################################################################################
if __name__ == "__main__":
    with redfish.redfish_client( base_url = base_url, username = user, password = Password ) as _redfishobj:  
        # Retrieve the Redfish root services from the standard location /redfish/v1/
        service_root = _redfishobj.get( "/redfish/v1/" )
        
        # Make sure the OData Expand query is supported by the remote iLO 5, at least for 1 level
        print("Odata supported queries:")
        print(json.dumps(service_root.dict['ProtocolFeaturesSupported'], indent=4) + "\n\n\n")
        
        # Retrieve the Chassis collection URI 
        chassis_collection_URI = _redfishobj.get( service_root.dict["Chassis"]["@odata.id"] )
        
        # For each chassis in the collection retrieve the Devices collection URI, 
        # extract their properties and print them
        for chassis_member in chassis_collection_URI.dict["Members"]:
            
            # Retrieve all the resources present in this chassis
            chassis_resources = _redfishobj.get( chassis_member["@odata.id"] )
            
            # Find the URI of the Devices resource. To be perfect, we should have found the 
            # location of the Oem/Hpe Devices from the Schema.
            chassis_devices_uri = chassis_resources.obj.Oem.Hpe.Links['Devices']['@odata.id']
            
            # Retrieve the URIs of the Devices as well as their properties using 
            # a single GET request and the $expand odata parameter
            chassis_device_members = _redfishobj.get( chassis_devices_uri + "?$expand=." )
            
            # Extract Devices properties from previous
            chassis_device_properties = chassis_device_members.obj['Members']
            
            # Print requested properties
            print ("Device Inventory list")
            for i in chassis_device_properties:
                print(i['Location'])
                print("\t" + i['Name'])
                print("\t" + str(i['ProductVersion']))
                print("\t" + str(i['FirmwareVersion']))
                print("\t" + str(i['Status']))
                print("\n")
              

Odata supported queries:
{
    "ExpandQuery": {
        "ExpandAll": false,
        "Levels": true,
        "Links": false,
        "MaxLevels": 1,
        "NoLinks": true
    },
    "FilterQuery": true,
    "OnlyMemberQuery": true,
    "SelectQuery": false
}

Device Inventory
PCI-E Slot 1
	Empty slot 1
	
	{'Current': {'VersionString': 'N/A'}}
	{'State': 'Absent'}


Embedded LOM
	Intel(R) I350 Gigabit Network Connection
	
	{'Current': {'VersionString': '1.2507.0'}}
	{'Health': None, 'State': None}


Embedded Device
	Embedded Video Controller
	
	{'Current': {'VersionString': '2.5'}}
	{'Health': 'OK', 'State': 'Enabled'}


Embedded Device
	Cannon Lake PCH-H AHCI Controller
	
	{'Current': {'VersionString': 'N/A'}}
	{'Health': 'OK', 'State': 'Enabled'}


